# 제품 이상 여부 판별 프로젝트

# 1. 데이터 & 라이브러리 불러오기

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.2 MB/s eta 0:00:00


In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.8 MB/s eta 0:00:00


In [ ]:
import os
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings; warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aimer본선/data/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aimer본선/data/test.csv')

### 데이터 읽어오기

In [ ]:
ROOT_DIR = "data"
RANDOM_STATE = 77

# Load data
train_data = pd.read_csv(os.path.join("data/train.csv"))
test_data = pd.read_csv(os.path.join("data/test.csv"))
print("train shape : ", train_data.shape)
print("test shape : ", test_data.shape)

train shape :  (40506, 468)
test shape :  (17361, 469)


In [ ]:
df_hand = pd.read_excel(os.path.join(ROOT_DIR, "hand_data.xlsx"))
df_hand.head(6)

,day,start_time,end_time,Dam Thickness - 1time / day,Fill thickness - 1time / day,Fill cure energy (mJ) - Every break time,Fill cure power (mW) - Every break time,Line Temp. - Every break time,Line humidity - Every break time,OCR gap - Line#1 stage1,OCR gap - Line#1 stage2,OCR gap - Line#1 stage3,OCR gap - Line #2 stage1,OCR gap - Line #2 stage2,OCR gap - Line #2 stage3
0,240319,08:00:00,10:30:00,454.0,193.0,1728.0,2919.0,23.4,56.6,1.2,1.0,1.2,1.2,0.9,1.4
1,240319,10:45:00,12:00:00,454.0,193.0,1754.0,2962.0,24.0,50.9,1.2,1.0,1.2,1.2,0.9,1.4
2,240319,13:00:00,15:00:00,454.0,193.0,1708.0,2887.0,24.5,49.6,1.2,1.0,1.2,1.2,0.9,1.4
3,240319,15:15:00,18:00:00,454.0,193.0,1751.0,2946.0,24.7,53.4,1.2,1.0,1.2,1.2,1.2,1.4
4,240319,18:30:00,20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,240320,08:00:00,10:30:00,481.0,212.0,1739.0,2875.0,24.2,55.6,1.2,1.0,1.2,1.2,1.2,1.4


# 2. 데이터 전처리 - 1

### Null값 확인

In [ ]:
train_data.isna().sum()

,0
Wip Line_Dam,0
Process Desc._Dam,0
Equipment_Dam,0
Model.Suffix_Dam,0
Workorder_Dam,0
...,...
Receip No Judge Value_Fill2,40506
WorkMode Collect Result_Fill2,0
WorkMode Unit Time_Fill2,40506
WorkMode Judge Value_Fill2,40506


### Null이 50%이상인 컬럼제거


In [ ]:
# Null 값이 포함된 컬럼만 추출
null_columns = train_data.columns[train_data.isna().any()]

# Null 값이 포함된 컬럼들의 null 값 개수 확인
null_counts = train_data[null_columns].isna().sum()

# Drop columns with more than half of the values missing
drop_cols = []
for column in train_data.columns:
    if (train_data[column].notnull().sum() // 2) < train_data[
        column
    ].isnull().sum():
        drop_cols.append(column)
train_data = train_data.drop(drop_cols, axis=1)
print("train shape : ", train_data.shape)

train shape :  (40506, 185)


In [ ]:
features = []
for col in train_data.columns:
    try:
        features.append(col)
    except:
        continue

In [ ]:
test_data = test_data[features]
print("test shape : ", test_data.shape)

test shape :  (17361, 185)


# 3. 데이터 전처리 - 2

### 다시 NULL값 확인


In [ ]:
# Null 값이 포함된 컬럼만 추출
null_columns = train_data.columns[train_data.isna().any()]

# Null 값이 포함된 컬럼들의 null 값 개수 확인
null_counts = train_data[null_columns].isna().sum()

null_counts

,0
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,12766
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1,12766
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2,12766


In [ ]:
# 컬럼 이름 설정
columns_of_interest = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# 각 컬럼의 유니크 값 찾기
for column in columns_of_interest:
    unique_values = train_data[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print()  # 빈 줄 추가

Unique values in column 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam':
[nan '550.3' 'OK' '162.4' '549' '549.5' '550' '548.5']

Unique values in column 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1':
[nan '838.4' 'OK' '837.7' '837.9' '838.2' '837.5']

Unique values in column 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2':
[nan '835.5' 'OK' '305']



NULL, 숫자형, 범주형 변수가 섞여있음 -> 데이터가 밀린것을 확인

### 데이터 밀려있는부분 전처리

#### Train - Dam 밀린부분 처리

In [ ]:
# Dam 안밀린부분 떼어내기
D0 = train_data.iloc[:, :25]

# Dam 밀린부분 작업해주기
Dam1 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] == "OK") |
                  (pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']))]
Dam2 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] != "OK") &
                  (~pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']))]

Dam_col = train_data.columns[25:78]

D1 = Dam1[Dam_col]
D1 = D1.iloc[:, 1:]
D1['new'] = 7
D1.columns = Dam_col
D2 = Dam2[Dam_col]
D = pd.concat([D1, D2])

#### Train - AutoClave 밀린부분 처리

In [ ]:
# Auto Clave는 안밀려있음
Auto = train_data.iloc[:, 78:98]

#### Train - Fill1 밀린부분 처리

In [ ]:
# Fill1 안밀린부분 작업해주기
F1_0 = train_data.iloc[:, 98:113]

# Fill1 밀린부분 작업해주기
Fill1_1 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] == "OK") |
               (pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))]
Fill1_2 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] != "OK") &
               (~pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))]

Fill1_col = train_data.columns[113:136]
F1_1 = Fill1_1[Fill1_col]
F1_1 = F1_1.iloc[:, 1:]
F1_1['new'] = 7
F1_1.columns = Fill1_col
F1_2 = Fill1_2[Fill1_col]
FF1 = pd.concat([F1_1, F1_2])

#### Train - Fill2 밀린부분 처리

In [ ]:
# Fill2 안밀린부분 작업해주기
F2_0 = train_data.iloc[:, 136:161]
F2_0
# Fill2 밀린부분 작업해주기
Fill2_1 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] == "OK") |
               (pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))]
Fill2_2 = train_data[(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] != "OK") &
               (~pd.isna(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))]

Fill2_col = train_data.columns[161:184]
F2_1 = Fill2_1[Fill2_col]
F2_1 = F2_1.iloc[:, 1:]
F2_1['new'] = 0
F2_1.columns = Fill2_col
F2_2 = Fill2_2[Fill2_col]
FF2 = pd.concat([F2_1, F2_2])

### Train data 병합

In [ ]:
# target
target = train_data.iloc[:, 184]

# 병합
train_data = pd.concat([D0, D, Auto, F1_0, FF1, F2_0, FF2, target], axis=1)

In [ ]:
# Null 값이 포함된 컬럼만 추출
null_columns = train_data.columns[train_data.isna().any()]

# Null 값이 포함된 컬럼들의 null 값 개수 확인
null_counts = train_data[null_columns].isna().sum()

null_counts

,0


NULL 값이 없는것을 확인

### Test data도 동일하게 처리

In [ ]:
# Dam 안밀린부분 떼어내기
D0 = test_data.iloc[:, :25]

# Dam 밀린부분 작업해주기
Dam1 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] == "OK") |
                  (pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']))]
Dam2 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] != "OK") &
                  (~pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']))]

Dam_col = test_data.columns[25:78]

D1 = Dam1[Dam_col]
D1 = D1.iloc[:, 1:]
D1['new'] = 7
D1.columns = Dam_col
D2 = Dam2[Dam_col]
D = pd.concat([D1, D2])

In [ ]:
# Auto Clave는 안밀려있음
Auto = test_data.iloc[:, 78:98]

In [ ]:
# Fill1 안밀린부분 작업해주기
F1_0 = test_data.iloc[:, 98:113]

# Fill1 밀린부분 작업해주기
Fill1_1 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] == "OK") |
               (pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))]
Fill1_2 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] != "OK") &
               (~pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']))]

Fill1_col = test_data.columns[113:136]
Fill1_col
F1_1 = Fill1_1[Fill1_col]
F1_1 = F1_1.iloc[:, 1:]
F1_1['new'] = 7
F1_1.columns = Fill1_col
F1_2 = Fill1_2[Fill1_col]
FF1 = pd.concat([F1_1, F1_2])

In [ ]:
# Fill2 안밀린부분 작업해주기
F2_0 = test_data.iloc[:, 136:161]
F2_0
# Fill2 밀린부분 작업해주기
Fill2_1 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] == "OK") |
               (pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))]
Fill2_2 = test_data[(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] != "OK") &
               (~pd.isna(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']))]

Fill2_col = test_data.columns[161:184]

F2_1 = Fill2_1[Fill2_col]
F2_1 = F2_1.iloc[:, 1:]
F2_1['new'] = 0
F2_1.columns = Fill2_col
F2_2 = Fill2_2[Fill2_col]
FF2 = pd.concat([F2_1, F2_2])

In [ ]:
# 병합
test_data = pd.concat([D0, D, Auto, F1_0, FF1, F2_0, FF2], axis=1)

In [ ]:
# Null 값이 포함된 컬럼만 추출
null_columns = test_data.columns[test_data.isna().any()]

# Null 값이 포함된 컬럼들의 null 값 개수 확인
null_counts = test_data[null_columns].isna().sum()

null_counts

,0


### 데이터 타입 통일

In [ ]:
# 숫자형과 문자열이 섞여있는 열을 찾고 숫자형으로 변환
for column in train_data.columns:
    if train_data[column].apply(lambda x: isinstance(x, (int, float))).any() and train_data[column].apply(lambda x: isinstance(x, str)).any():
        train_data[column] = pd.to_numeric(train_data[column], errors='coerce')

In [ ]:
# 숫자형과 문자열이 섞여있는 열을 찾고 숫자형으로 변환
for column in test_data.columns:
    if test_data[column].apply(lambda x: isinstance(x, (int, float))).any() and test_data[column].apply(lambda x: isinstance(x, str)).any():
        test_data[column] = pd.to_numeric(test_data[column], errors='coerce')

# 4. 데이터 전처리 - 3

In [ ]:
# 불필요한 변수 제거
def variable_cleanup(data):
    # Workorder
    data_cleaned = data.drop(columns=['Workorder_Fill1', 'Workorder_AutoClave', 'Workorder_Fill2'])
    data_cleaned = data_cleaned.rename(columns={'Workorder_Dam': 'Workorder'})

    # Model.Suffix
    data_cleaned = data_cleaned.drop(columns=['Model.Suffix_Fill1', 'Model.Suffix_AutoClave', 'Model.Suffix_Fill2'])
    data_cleaned = data_cleaned.rename(columns={'Model.Suffix_Dam': 'Model.Suffix'})

    # Wip Line, Process Desc, Insp. Seq No., Insp Judge Code, WorkMode , Equipment_AutoClave
    data_cleaned = data_cleaned.drop(columns=[
        'Wip Line_Dam', 'Wip Line_Fill1', 'Wip Line_AutoClave', 'Wip Line_Fill2',
        'Process Desc._Dam', 'Process Desc._Fill1', 'Process Desc._AutoClave', 'Process Desc._Fill2',
        'Insp. Seq No._Dam', 'Insp. Seq No._Fill1', 'Insp. Seq No._AutoClave', 'Insp. Seq No._Fill2',
        'Insp Judge Code_Dam', 'Insp Judge Code_Fill1', 'Insp Judge Code_AutoClave', 'Insp Judge Code_Fill2',
        'WorkMode Collect Result_Dam', 'WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2',
        'Equipment_AutoClave'
    ])

    # unique value = 0 또는 OK (only, 좌표 조합은 제외)
    data_cleaned = data_cleaned.drop(columns=[
        'DISCHARGED SPEED OF RESIN Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2',
        'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill2',
        'Dispense Volume(Stage1) Collect Result_Fill2', 'Dispense Volume(Stage2) Collect Result_Fill2', 'Dispense Volume(Stage3) Collect Result_Fill2',
        '1st Pressure Judge Value_AutoClave', '2nd Pressure Judge Value_AutoClave', '3rd Pressure Judge Value_AutoClave'
    ])

    # 새로운 column drop
    data_cleaned = data_cleaned.drop(columns=[
        'CURE STANDBY POSITION X Collect Result_Dam', 'CURE STANDBY POSITION Z Collect Result_Dam', 'CURE STANDBY POSITION Θ Collect Result_Dam',
        'CURE START POSITION Z Collect Result_Dam',
        'HEAD Standby Position X Collect Result_Dam', 'HEAD Standby Position Y Collect Result_Dam', 'HEAD Standby Position Z Collect Result_Dam',
        'Head Clean Position X Collect Result_Dam', 'Head Clean Position Y Collect Result_Dam',
        'Head Purge Position X Collect Result_Dam', 'Head Purge Position Y Collect Result_Dam',
        'Head Zero Position X Collect Result_Dam',
        'HEAD Standby Position X Collect Result_Fill1','HEAD Standby Position Y Collect Result_Fill1','HEAD Standby Position Z Collect Result_Fill1',
        'Head Clean Position X Collect Result_Fill1','Head Clean Position Y Collect Result_Fill1','Head Clean Position Z Collect Result_Fill1',
        'Head Purge Position X Collect Result_Fill1','Head Purge Position Y Collect Result_Fill1',
        'CURE END POSITION Θ Collect Result_Fill2',
        'CURE STANDBY POSITION X Collect Result_Fill2', 'CURE STANDBY POSITION Θ Collect Result_Fill2',
        'CURE START POSITION Θ Collect Result_Fill2',
        'HEAD Standby Position X Collect Result_Fill2','HEAD Standby Position Y Collect Result_Fill2','HEAD Standby Position Z Collect Result_Fill2',
        'Head Clean Position X Collect Result_Fill2','Head Clean Position Y Collect Result_Fill2','Head Clean Position Z Collect Result_Fill2',
        'Head Purge Position X Collect Result_Fill2','Head Purge Position Y Collect Result_Fill2'
    ])

    return data_cleaned

train_data = variable_cleanup(train_data); test_data = variable_cleanup(test_data)
print(train_data.shape); print(test_data.shape)

(40506, 117)
(17361, 116)


# 5. 데이터 전처리 - 4

In [ ]:
# durations
train_data['Collect Date_Dam'] = pd.to_datetime(train_data['Collect Date_Dam'])
train_data['Collect Date_AutoClave'] = pd.to_datetime(train_data['Collect Date_AutoClave'])
train_data['Collect Date_Fill1'] = pd.to_datetime(train_data['Collect Date_Fill1'])
train_data['Collect Date_Fill2'] = pd.to_datetime(train_data['Collect Date_Fill2'])
train_data['durations']=train_data['Collect Date_AutoClave'] - train_data['Collect Date_Dam']

test_data['Collect Date_Dam'] = pd.to_datetime(test_data['Collect Date_Dam'])
test_data['Collect Date_AutoClave'] = pd.to_datetime(test_data['Collect Date_AutoClave'])
test_data['Collect Date_Fill1'] = pd.to_datetime(test_data['Collect Date_Fill1'])
test_data['Collect Date_Fill2'] = pd.to_datetime(test_data['Collect Date_Fill2'])
test_data['durations']=test_data['Collect Date_AutoClave'] - test_data['Collect Date_Dam']

# 각 공정 간의 시간 차이 계산 (분 단위)
train_data['time_diff_Dam_to_Fill1'] = (train_data['Collect Date_Fill1'] - train_data['Collect Date_Dam']).dt.total_seconds() / 60
train_data['time_diff_Fill1_to_Fill2'] = (train_data['Collect Date_Fill2'] - train_data['Collect Date_Fill1']).dt.total_seconds() / 60
train_data['time_diff_Fill2_to_AutoClave'] = (train_data['Collect Date_AutoClave'] - train_data['Collect Date_Fill2']).dt.total_seconds() / 60

# 각 공정 간의 시간 차이 계산 (분 단위)
test_data['time_diff_Dam_to_Fill1'] = (test_data['Collect Date_Fill1'] - test_data['Collect Date_Dam']).dt.total_seconds() / 60
test_data['time_diff_Fill1_to_Fill2'] = (test_data['Collect Date_Fill2'] - test_data['Collect Date_Fill1']).dt.total_seconds() / 60
test_data['time_diff_Fill2_to_AutoClave'] = (test_data['Collect Date_AutoClave'] - test_data['Collect Date_Fill2']).dt.total_seconds() / 60

# 각 공정별 HH:MM 변수
train_data['time_Dam'] = train_data['Collect Date_Dam'].dt.strftime('%H:%M')
test_data['time_Dam'] = test_data['Collect Date_Dam'].dt.strftime('%H:%M')
train_data['time_AutoClave'] = train_data['Collect Date_AutoClave'].dt.strftime('%H:%M')
test_data['time_AutoClave'] = test_data['Collect Date_AutoClave'].dt.strftime('%H:%M')
train_data['time_Fill1'] = train_data['Collect Date_Fill1'].dt.strftime('%H:%M')
test_data['time_Fill1'] = test_data['Collect Date_Fill1'].dt.strftime('%H:%M')
train_data['time_Fill2'] = train_data['Collect Date_Fill2'].dt.strftime('%H:%M')
test_data['time_Fill2'] = test_data['Collect Date_Fill2'].dt.strftime('%H:%M')

### Collect Date drop

In [ ]:
train_data = train_data.drop(columns = ['Collect Date_Dam', 'Collect Date_AutoClave', 'Collect Date_Fill1', 'Collect Date_Fill2'])
test_data = test_data.drop(columns = ['Collect Date_Dam', 'Collect Date_AutoClave', 'Collect Date_Fill1', 'Collect Date_Fill2'])

# 6. Tact time 제외하고 모두 범주형 처리

In [ ]:
# 범주형 및 연속형 변수 정의
numerical_cols = ['Machine Tact time Collect Result_Fill2', 'Machine Tact time Collect Result_Fill1', 'Machine Tact time Collect Result_Dam']
categorical_cols = list(set(train_data.columns) - set(numerical_cols) - set(['target']))
target_col = 'target'

len(categorical_cols)

117

In [ ]:
len(numerical_cols) + len(categorical_cols)

120

# 4. 모델링

### Train - Validation 구분

In [ ]:
# train set, validation set 구분
train_data[categorical_cols] = train_data[categorical_cols].astype('str')
test_data[categorical_cols] = test_data[categorical_cols].astype('str')
train_data['target'] = train_data['target'].apply(lambda x: 1 if x == 'AbNormal' else 0)

X = train_data.drop('target', axis=1)
y = train_data['target']

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import optuna

In [ ]:
X.head()

,Equipment_Dam,Model.Suffix,Workorder,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam,Head Clean Position Z Collect Result_Dam,Head Purge Position Z Collect Result_Dam,Head Zero Position Y Collect Result_Dam,Head Zero Position Z Collect Result_Dam,Machine Tact time Collect Result_Dam,PalletID Collect Result_Dam,Production Qty Collect Result_Dam,Receip No Collect Result_Dam,Stage1 Circle1 Distance Speed Collect Result_Dam,Stage1 Circle2 Distance Speed Collect Result_Dam,Stage1 Circle3 Distance Speed Collect Result_Dam,Stage1 Circle4 Distance Speed Collect Result_Dam,Stage1 Line1 Distance Speed Collect Result_Dam,Stage1 Line2 Distance Speed Collect Result_Dam,Stage1 Line3 Distance Speed Collect Result_Dam,Stage1 Line4 Distance Speed Collect Result_Dam,Stage2 Circle1 Distance Speed Collect Result_Dam,Stage2 Circle2 Distance Speed Collect Result_Dam,Stage2 Circle3 Distance Speed Collect Result_Dam,Stage2 Circle4 Distance Speed Collect Result_Dam,Stage2 Line1 Distance Speed Collect Result_Dam,Stage2 Line2 Distance Speed Collect Result_Dam,Stage2 Line3 Distance Speed Collect Result_Dam,Stage2 Line4 Distance Speed Collect Result_Dam,Stage3 Circle1 Distance Speed Collect Result_Dam,...,DISCHARGED SPEED OF RESIN Collect Result_Fill1,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1,Dispense Volume(Stage1) Collect Result_Fill1,Dispense Volume(Stage2) Collect Result_Fill1,Dispense Volume(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1,Head Purge Position Z Collect Result_Fill1,Machine Tact time Collect Result_Fill1,PalletID Collect Result_Fill1,Production Qty Collect Result_Fill1,Receip No Collect Result_Fill1,Equipment_Fill2,CURE END POSITION X Collect Result_Fill2,CURE END POSITION Z Collect Result_Fill2,CURE SPEED Collect Result_Fill2,CURE STANDBY POSITION Z Collect Result_Fill2,CURE START POSITION X Collect Result_Fill2,CURE START POSITION Z Collect Result_Fill2,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Resul

In [ ]:
def CAT(params, seed):

    # 데이터 분할
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y, random_state=seed)

    # 모델 초기화 및 학습(모델 시드는 77로 고정)
    model = CatBoostClassifier(**params, random_state=77, verbose=0, cat_features=categorical_cols, early_stopping_rounds=150)
    # 최종 검증 데이터셋에서 모델 성능 평가
    model.fit(X_train, y_train)
    y_pred_final = model.predict(X_val)

    accuracy = accuracy_score(y_val, y_pred_final) ; precision = precision_score(y_val, y_pred_final)
    recall = recall_score(y_val, y_pred_final) ; f1_final = f1_score(y_val, y_pred_final)
    conf_matrix = confusion_matrix(y_val, y_pred_final)

    # 결과 출력
    print(f"Accuracy: {accuracy:.4f}") ; print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}") ; print(f"F1 Score: {f1_final:.4f}")
    print("\nConfusion Matrix:") ; print(conf_matrix)

    # 변수 중요도 계산
    feature_importances = model.get_feature_importance()

    # 변수 이름과 중요도 및 nunique 값을 zip으로 묶은 후 중요도 기준으로 내림차순 정렬
    sorted_importances = sorted(zip(X_train.columns, feature_importances, X_train.nunique()), key=lambda x: x[1], reverse=True)

    # 상위 10개 변수 중요도 및 nunique 값 출력
    print("\nTop 10 Feature Importances:")
    for feature_name, importance, nunique in sorted_importances[:10]:
        print(f"Feature: {feature_name}, Importance: {importance:.4f}, Unique Values: {nunique}")

    return f1_final

In [ ]:
def objective(trial):
    # Hyperparameter search space
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.08, step=0.001),
        'depth': trial.suggest_int('depth', 4, 8),
        'n_estimators': trial.suggest_int('n_estimators', 500, 1100),
        'class_weights': [1, trial.suggest_float('class_weights', 7.5, 10, step=0.01)],
        'l2_leaf_reg' : trial.suggest_float('l2_leaf_reg', 3, 10, step=0.1),
        'task_type' : 'GPU'
        #,
        #'devices' : '0'
    }

    # 시드 10개 중 무작위 선택
    seed = 200

    # CAT 함수를 사용해 F1 스코어 반환
    f1_score = CAT(params, seed)

    return f1_score

# Optuna study를 생성하고 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# 최적화된 파라미터 출력
print("Best parameters found: ", study.best_params)

[I 2024-09-28 17:17:18,479] A new study created in memory with name: no-name-b4420113-fd0a-4287-8411-12225bae3a75


Accuracy: 0.9077
Precision: 0.2467
Recall: 0.2879
F1 Score: 0.2657

Confusion Matrix:
[[10827   620]
 [  502   203]]


[I 2024-09-28 17:18:35,737] Trial 0 finished with value: 0.2657068062827225 and parameters: {'learning_rate': 0.067, 'depth': 4, 'n_estimators': 771, 'class_weights': 9.870000000000001, 'l2_leaf_reg': 5.4}. Best is trial 0 with value: 0.2657068062827225.



Top 10 Feature Importances:
Feature: durations, Importance: 29.6490, Unique Values: 39
Feature: Workorder, Importance: 6.2741, Unique Values: 663
Feature: time_diff_Fill2_to_AutoClave, Importance: 2.6404, Unique Values: 33
Feature: 1st Pressure Collect Result_AutoClave, Importance: 2.2134, Unique Values: 27
Feature: Machine Tact time Collect Result_Fill2, Importance: 2.0596, Unique Values: 76
Feature: DISCHARGED SPEED OF RESIN Collect Result_Fill1, Importance: 2.0390, Unique Values: 4
Feature: HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam, Importance: 1.8381, Unique Values: 37
Feature: Head Clean Position Z Collect Result_Dam, Importance: 1.5329, Unique Values: 5
Feature: 3rd Pressure Collect Result_AutoClave, Importance: 1.4949, Unique Values: 66
Feature: time_Dam, Importance: 1.4832, Unique Values: 135
Accuracy: 0.9184
Precision: 0.2723
Recall: 0.2426
F1 Score: 0.2566

Confusion Matrix:
[[10990   457]
 [  534   171]]


[I 2024-09-28 17:20:15,194] Trial 1 finished with value: 0.2565641410352588 and parameters: {'learning_rate': 0.056999999999999995, 'depth': 7, 'n_estimators': 803, 'class_weights': 8.74, 'l2_leaf_reg': 5.1}. Best is trial 0 with value: 0.2657068062827225.


...


Top 10 Feature Importances:
Feature: durations, Importance: 26.0213, Unique Values: 39
Feature: Workorder, Importance: 7.9610, Unique Values: 663
Feature: Machine Tact time Collect Result_Fill2, Importance: 2.7997, Unique Values: 76
Feature: Machine Tact time Collect Result_Dam, Importance: 2.3839, Unique Values: 429
Feature: DISCHARGED SPEED OF RESIN Collect Result_Fill1, Importance: 2.3453, Unique Values: 4
Feature: 1st Pressure Collect Result_AutoClave, Importance: 2.1572, Unique Values: 27
Feature: HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam, Importance: 2.0331, Unique Values: 14
Feature: time_diff_Fill2_to_AutoClave, Importance: 1.6869, Unique Values: 33
Feature: time_Fill1, Importance: 1.6549, Unique Values: 134
Feature: Head Purge Position Z Collect Result_Dam, Importance: 1.6183, Unique Values: 3
Accuracy: 0.9096
Precision: 0.2497
Recall: 0.2780
F1 Score: 0.2631

Confusion Matrix:
[[10858   589]
 [  509   196]]


[I 2024-09-28 17:31:36,089] Trial 8 finished with value: 0.2630872483221477 and parameters: {'learning_rate': 0.024, 'depth': 7, 'n_estimators': 788, 'class_weights': 9.780000000000001, 'l2_leaf_reg': 8.9}. Best is trial 7 with value: 0.28949858088930935.



Top 10 Feature Importances:
Feature: durations, Importance: 20.6679, Unique Values: 39
Feature: Workorder, Importance: 6.2920, Unique Values: 663
Feature: 1st Pressure Collect Result_AutoClave, Importance: 3.6523, Unique Values: 27
Feature: Machine Tact time Collect Result_Fill2, Importance: 2.6653, Unique Values: 76
Feature: Machine Tact time Collect Result_Dam, Importance: 2.5373, Unique Values: 429
Feature: Chamber Temp. Collect Result_AutoClave, Importance: 2.1162, Unique Values: 25
Feature: 2nd Pressure Collect Result_AutoClave, Importance: 2.0752, Unique Values: 67
Feature: 3rd Pressure Collect Result_AutoClave, Importance: 1.9398, Unique Values: 66
Feature: PalletID Collect Result_Fill2, Importance: 1.8755, Unique Values: 16
Feature: HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam, Importance: 1.8511, Unique Values: 37
Accuracy: 0.9206
Precision: 0.2917
Recall: 0.2582
F1 Score: 0.2739

Confusion Matrix:
[[11005   442]
 [  523   182]]


[I 2024-09-28 17:32:20,291] Trial 9 finished with value: 0.27389014296463504 and parameters: {'learning_rate': 0.077, 'depth': 5, 'n_estimators': 630, 'class_weights': 8.83, 'l2_leaf_reg': 5.4}. Best is trial 7 with value: 0.28949858088930935.



Top 10 Feature Importances:
Feature: durations, Importance: 20.5997, Unique Values: 39
Feature: Workorder, Importance: 5.0562, Unique Values: 663
Feature: Machine Tact time Collect Result_Fill2, Importance: 2.8889, Unique Values: 76
Feature: PalletID Collect Result_Dam, Importance: 2.6982, Unique Values: 16
Feature: 1st Pressure Collect Result_AutoClave, Importance: 2.6234, Unique Values: 27
Feature: DISCHARGED SPEED OF RESIN Collect Result_Fill1, Importance: 2.0168, Unique Values: 4
Feature: Machine Tact time Collect Result_Dam, Importance: 1.9286, Unique Values: 429
Feature: 3rd Pressure Collect Result_AutoClave, Importance: 1.8999, Unique Values: 66
Feature: time_Dam, Importance: 1.8230, Unique Values: 135
Feature: Model.Suffix, Importance: 1.7815, Unique Values: 7
Best parameters found:  {'learning_rate': 0.02, 'depth': 5, 'n_estimators': 960, 'class_weights': 7.77, 'l2_leaf_reg': 5.2}


# 각 파라미터 조합 검증

In [ ]:
params =

In [ ]:
# Seed 범위 설정
seeds = [17, 27, 37, 47, 57, 67, 77, 87, 97, 107, 10, 80]

# F1 Score를 기록할 리스트 초기화
f1_scores = []
proba_results = {}
pred_results = {}

# Seed 값을 변경하며 모델 학습 및 평가
for seed in seeds:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y, random_state=seed)

    # 모델 초기화 및 학습
    model = CatBoostClassifier(**params, random_state=77, verbose=0, cat_features=categorical_cols, early_stopping_rounds = 150)
    model.fit(X_train, y_train)

    # 예측 및 평가
    y_pred = model.predict(X_val) ; y_proba = model.predict_proba(X_val)
    accuracy = accuracy_score(y_val, y_pred) ; precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred) ; f1 = f1_score(y_val, y_pred) ; conf_matrix = confusion_matrix(y_val, y_pred)

    # 결과 출력
    print(f"Accuracy: {accuracy:.4f}"); print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}"); print(f"F1 Score: {f1:.4f}")
    print("\nConfusion Matrix:"); print(conf_matrix) ; print(pd.DataFrame(y_pred).value_counts())

    # F1 Score 기록
    f1_scores.append(f1)
    proba_results[seed] = y_proba
    pred_results[seed] = y_pred

    # 현재 seed에 대한 결과 출력
    print(f"Seed: {seed}, F1 Score: {f1:.4f}")

# F1 Score 플롯 그리기
plt.figure(figsize=(8, 5)) ; plt.plot(seeds, f1_scores, marker='o', linestyle='-', color='b')
plt.title('F1 Score for Different Seeds')
plt.xlabel('Seed') ; plt.ylabel('F1 Score') ; plt.grid(True) ; plt.show()
print(pd.DataFrame(f1_scores).mean())

### K-fold로 확인

In [ ]:
# 함수정의
def catboost_fold(params, seed, n_splits=5):

    # StratifiedKFold 설정
    kf = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state=seed)

    fold = 1
    for train_index, val_index in kf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # 모델 초기화 및 훈련
        model = CatBoostClassifier(**params, random_state=77, verbose=0, cat_features=categorical_cols)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=150)

        # 검증 데이터에 대한 예측
        y_val_probs = model.predict_proba(X_val)[:, 1]
        y_pred = (y_val_probs >= 0.5).astype(int)

        # 성능 지표 계산
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        conf_matrix = confusion_matrix(y_val, y_pred)

        # 결과 출력
        print(f"Fold {fold}:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print("\nConfusion Matrix:")
        print(conf_matrix)
        print("\n")

        fold += 1

catboost_fold(params,17,n_splits=5)

## Voting

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
params1 = {'learning_rate': 0.05857910198594344, 'depth': 7, 'n_estimators': 561, 'class_weights': [1, 7.1], 'l2_leaf_reg': 9}
params2 = {'learning_rate': 0.048934679784453365, 'depth': 6, 'n_estimators': 827, 'class_weights': [1, 8.3], 'l2_leaf_reg': 12}
params3 = {'learning_rate': 0.05184429466563718, 'depth': 9, 'n_estimators': 950, 'class_weights': [1, 7.4], 'l2_leaf_reg': 3}
params4 = {'learning_rate': 0.07797672575412745, 'depth': 5, 'n_estimators': 869, 'class_weights': [1, 7.4], 'l2_leaf_reg': 6}

In [ ]:
model1 = CatBoostClassifier(**params1, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150, task_type='GPU', devices ='0')
model2 = CatBoostClassifier(**params2, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150, task_type='GPU', devices ='0')
model3 = CatBoostClassifier(**params3, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150, task_type='GPU', devices ='0')
model4 = CatBoostClassifier(**params4, cat_features=categorical_cols, random_state=77, verbose=0, early_stopping_rounds  = 150, task_type='GPU', devices ='0')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y, random_state=77)

In [ ]:
# Seed 범위 설정
seeds = [17, 27, 37, 47, 57, 67, 77, 87, 97, 107, 10, 80]

# F1 Score를 기록할 리스트 초기화
f1_scores = []
proba_results = {}
pred_results = {}

# Seed 값을 변경하며 모델 학습 및 평가
for seed in seeds:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y, random_state=seed)

    # Voting
    voting_clf = VotingClassifier(
    estimators=[('cat1', model1), ('cat2', model2), ('cat3', model3),
               ('cat4', model4)],
    voting='soft'  # soft, hard
            )

    # VotingClassifier 학습
    voting_clf.fit(X_train, y_train)

    # 예측 및 평가
    y_pred = voting_clf.predict(X_val) ; y_proba = voting_clf.predict_proba(X_val)
    accuracy = accuracy_score(y_val, y_pred) ; precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred) ; f1 = f1_score(y_val, y_pred) ; conf_matrix = confusion_matrix(y_val, y_pred)

    # 결과 출력
    print(f"Accuracy: {accuracy:.4f}"); print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}"); print(f"F1 Score: {f1:.4f}")
    print("\nConfusion Matrix:"); print(conf_matrix) ; print(pd.DataFrame(y_pred).value_counts())

    # F1 Score 기록
    f1_scores.append(f1)
    proba_results[seed] = y_proba
    pred_results[seed] = y_pred

    # 현재 seed에 대한 결과 출력
    print(f"Seed: {seed}, F1 Score: {f1:.4f}")

# 최대 및 최소 F1 점수와 해당 시드 출력
max_f1_score = max(f1_scores)
min_f1_score = min(f1_scores)
max_f1_seed = seeds[f1_scores.index(max_f1_score)]
min_f1_seed = seeds[f1_scores.index(min_f1_score)]

print(f"\nMax F1 Score: {max_f1_score:.4f} at Seed: {max_f1_seed}")
print(f"Min F1 Score: {min_f1_score:.4f} at Seed: {min_f1_seed}")

# F1 Score 평균 출력
mean_f1_score = pd.DataFrame(f1_scores).mean()
print(f"\nMean F1 Score: {mean_f1_score[0]:.4f}")

# F1 Score 플롯 그리기
plt.figure(figsize=(8, 5)) ; plt.plot(seeds, f1_scores, marker='o', linestyle='-', color='b')
plt.title('F1 Score for Different Seeds')
plt.xlabel('Seed') ; plt.ylabel('F1 Score') ; plt.grid(True) ; plt.show()

In [ ]:
# 함수정의
def voting_fold(seed, n_splits=5):

    # StratifiedKFold 설정
    kf = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state=seed)

    fold = 1
    for train_index, val_index in kf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # 모델 초기화 및 훈련
        voting_clf = VotingClassifier(
        estimators=[('cat1', model1), ('cat2', model2), ('cat3', model3),
                   ('cat4', model4), ('cat5', model5), ('cat6', model6)],
                    voting='soft'  # soft, hard
                        )

        # VotingClassifier 학습
        voting_clf.fit(X_train, y_train)

        # 예측 및 평가
        y_pred = voting_clf.predict(X_val) ; y_proba = voting_clf.predict_proba(X_val)

        # 성능 지표 계산
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        conf_matrix = confusion_matrix(y_val, y_pred)

        # 결과 출력
        print(f"Fold {fold}:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print("\nConfusion Matrix:")
        print(conf_matrix)
        print("\n")

        fold += 1

voting_fold(17,n_splits=5)

# 5. Submission

In [ ]:
# VotingClassifier 정의 (Soft Voting 사용)
voting_clf = VotingClassifier(
    estimators=[('cat1', model1), ('cat2', model2), ('cat3', model3),
               ('cat4', model4)],
    voting='soft'  # soft, hard
            )

# VotingClassifier 학습
voting_clf.fit(X, y)
# 예측 및 성능 평가
y_pred = voting_clf.predict(test_data)
pd.DataFrame(y_pred).value_counts()

In [ ]:
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
df_sub["target"].value_counts()

In [ ]:
#제출 파일 저장
df_sub.to_csv("submission.csv", index=False)